In [2]:
def sdata(ticker, period):
    import yfinance as yf
    data=yf.download(ticker,period=period)
    data.columns = [i.lower() for i in data.columns]
    return data

In [3]:
def Bollinger(data):
    import pandas as pd
    from talib.abstract import BBANDS

    #set Bollinger band
    bb_time = 10
    data = data.join(BBANDS(data, timeperiod = bb_time ))

    #set position and trade record
    position=0
    trade=pd.DataFrame()

    #run strategy and record buy-cover points
    for i in range(bb_time+1,data.shape[0]-1):
        c_time=data.index[i]
        c_close=data.loc[c_time,'close']
        c_upper=data.loc[c_time,'upperband']
        c_lower=data.loc[c_time,'lowerband']
        n_time=data.index[i+1]
        n_open=data.loc[n_time,'open']
        if position ==0  :
            #enter buy position if stock price > upper band
            if c_close > c_upper :
                position = 1  
                order_time=n_time
                order_price=n_open
                data.loc[n_time,'buy_order']=n_open
        #cover the position if the stock price falls below lower band
        elif position ==1 and c_close < c_lower :
            position = 0
            cover_time=n_time
            cover_price=n_open
            data.loc[n_time,'buy_cover']=n_open
            trade=trade.append(pd.Series(['Buy', order_time, order_price, cover_time, cover_price]),ignore_index=True)
    return [data, trade]

In [4]:
def ChartOrder(data,  addp = []):
    import mplfinance as mpf
    mcolor = mpf.make_marketcolors(up='r',down ='g', inherit = True)
    mstyle = mpf.make_mpf_style(base_mpf_style = 'starsandstripes', marketcolors = mcolor)
    
    if 'buy_order' in data.columns:
        addp.append(mpf.make_addplot(data['buy_order'], type= 'scatter', color = 'blue', marker = '^', markersize = 50))
    if 'buy_cover' in data.columns:
        addp.append(mpf.make_addplot(data['buy_cover'], type= 'scatter', color = 'orange', marker = 'v', markersize = 50))

    mpf.plot(data,type = 'candle', style = mstyle, addplot = addp)

In [5]:
def performance(trade):
    
    trade_num = trade.shape[0]
    
    #Calculate win/loss trades
    earn_trade = trade[(trade[4] - trade[2])>0]
    loss_trade = trade[(trade[4] - trade[2])<0]

    #Calculate average return
    avg_earn = ((earn_trade[4] - earn_trade[2])/earn_trade[2]).mean()*100
    avg_loss = ((loss_trade[4] - loss_trade[2])/loss_trade[2]).mean()*100

    #Calculate win ratio
    win_ratio = round(earn_trade.shape[0]/trade_num, 2)
    loss_ratio = 1 - win_ratio

    #Calculate Earn-loss ratio
    earn_loss = abs(avg_earn/avg_loss)

    #Expectation
    expectation = (win_ratio * earn_loss) - loss_ratio 

    print(
    'Executions:', trade_num,
    '\n Average Return(%):', round(((trade[4]-trade[2])/trade[2]).mean()*100,4),
    '\n Win rate(%):', round(earn_trade.shape[0]/trade_num, 2),
    '\n Profit-loss Ratio:', round(earn_loss,2),
    '\n Expectation:', round(expectation,2) 
    )

In [ ]:
import mplfinance as mpf
stock = sdata('MSFT', '3y')
[stock, trade] = Bollinger(stock)

#set up Plot with Bollinger bands
addp=[]
addp.append(mpf.make_addplot(stock['upperband']))
addp.append(mpf.make_addplot(stock['middleband']))
addp.append(mpf.make_addplot(stock['lowerband']))

#add in the trade execution indicators
ChartOrder(stock,addp)

#Performance Report
performance(trade)

In [ ]:
#point of improvement
'''
1. allow more tickers - mimick the composition of mutual funds or ETFs
2. allow more flexible time frame
3. Combine stock portfolio picking with market entering strategy - no idea 
'''
